<a href="https://colab.research.google.com/github/hasan-nn/BirdCliff22_solution/blob/main/notebook_gdrive2colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

importing the data sample from google drive:

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!ls -la
!cp '/content/gdrive/MyDrive/WHU_sample' './' -r
# !cat '/content/gdrive/My Drive/coco.py'
# import sys
# sys.path.append('/content/gdrive/My Drive')


total 24
drwxr-xr-x 1 root root 4096 Jun 13 18:44 .
drwxr-xr-x 1 root root 4096 Jun 13 18:40 ..
drwxr-xr-x 4 root root 4096 Jun  1 13:49 .config
drwx------ 5 root root 4096 Jun 13 18:43 gdrive
drwxr-xr-x 1 root root 4096 Jun  1 13:50 sample_data
drwx------ 5 root root 4096 Jun 13 18:50 WHU_sample


In [2]:
#!cp  '/content/gdrive/MyDrive/WHU_sample.tar.xz' './'
#!tar -xf './WHU_sample.tar.xz'

In [ ]:
# import sys
# sys.path.insert(0, "/content/gdrive/MyDrive")

Downloading detectron2:

In [1]:
!python -m pip install pyyaml==5.1
# Detectron2 has not released pre-built binaries for the latest pytorch (https://github.com/facebookresearch/detectron2/issues/4053)
# so we install from source instead. This takes a few minutes.
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

# Install pre-built detectron2 that matches pytorch version, if released:
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
#!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/{CUDA_VERSION}/{TORCH_VERSION}/index.html

# exit(0)  # After installation, you may need to "restart runtime" in Colab. This line can also restart runtime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 274 kB 37.1 MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp37-cp37m-linux_x86_64.whl size=44092 sha256=a0da17d084ff2f2c923e92bd2363ffe93724c17c3bbebe912de3ba8b785d2102
  Stored in directory: /root/.cache/pip/wheels/77/f5/10/d00a2bd30928b972790053b5de0c703ca87324f3fead0f2fd9
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-0p3l_vv2
  Running command git clone -q https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-0p3l_vv2
     |████████████████████████████████| 50 kB 8.0 MB/s 
     |████████████████████████████████| 79 kB

In [2]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
torch:  1.11 ; cuda:  cu113
detectron2: 0.6


In [3]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

downloading geopandas:

In [4]:
!pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 29.0 MB/s 
     |████████████████████████████████| 16.7 MB 55.5 MB/s 
     |████████████████████████████████| 6.3 MB 25.4 MB/s 


importing some libraries:

In [5]:
import os
import torch
import numpy as np
import cv2 as cv
from google.colab.patches import cv2_imshow
import geopandas as gp
import shapely
import json
from shapely.geometry import Polygon

**forming the coco dataset:**

In [6]:
dataset = dict() #initializing the dataset
#for explanation of how to form a coco dataset
#https://towardsdatascience.com/how-to-work-with-object-detection-datasets-in-coco-format-9bf4fb5848a4


#filling the categories field

dataset["categories"] = [{
            "id": 0,
            "name": "building",
            "supercategory": "none"
        }]


path_images = "/content/WHU_sample/train/image" #image path
k = os.listdir(path_images)
#initializing the images field and annotation fields which should be a list
dataset["images"] = list()
dataset["annotations"] = list()


path = "/content/WHU_sample/train/shapes"
k = os.listdir(path)
id = 0
id_image = 0
category_id = 0



#this function fills the annotation field with needed data

def call_g(g) :
  #point, linestring, multilinestring are all skew data which don't represen buildings
  if g.type in ['Point', 'LineString', 'MultiLineString']:
    return
  dataset["annotations"].append(dict()) #each element of this list is a dict containing the following fields
  #"id", "image_id", "category_id", "bbox", "area", "segmentation", "iscrowd"
  dataset["annotations"][id]["id"] = id
  dataset["annotations"][id]["image_id"] = id_image #the id of the image it corresponds to
  dataset["annotations"][id]["category_id"] = 0
  dataset["annotations"][id]["bbox"] = list() 
  #bound box 4 xy,xy
  l = g.bounds
  dataset["annotations"][id]["bbox"].append(l[0])
  dataset["annotations"][id]["bbox"].append(l[1])
  dataset["annotations"][id]["bbox"].append(l[2])
  dataset["annotations"][id]["bbox"].append(l[3])

  dataset["annotations"][id]["area"] = g.area

    #segmentation

  dataset["annotations"][id]["segmentation"] = list()
  dataset["annotations"][id]["segmentation"].append(list())
  x_list = list( g.exterior.coords.xy[0] )
  y_list = list( g.exterior.coords.xy[1] )
  for x, y in zip(x_list, y_list):
    dataset["annotations"][id]["segmentation"][0].append(x)
    dataset["annotations"][id]["segmentation"][0].append(y)
  
  dataset["annotations"][id]["iscrowd"] = 0

#let's loop on the dataset:
#id_image inititalized at zero
#the image field contains: "id", "file_name", "height", "width"
for i in k:
  dataset["images"].append(dict())
  dataset["images"][id_image]["id"] = id_image
  dataset["images"][id_image]["file_name"] = path_images + "/" + i + ".tif"
  l = cv.imread(path_images + "/" + i + ".tif")
  dataset["images"][id_image]["height"],dataset["images"][id_image]["width"] = l.shape[:2]
  

  #each image contains several shapes
  #we will loop over them
  gdf = gp.read_file(path + "/"+i)  #reading the gp dataframe from the directory
  for g in gdf["geometry"]:
    #multipolygon contains several polygons we should loop over each
    if g.type is "MultiPolygon":
      for pol in g:
        call_g(pol)
        id = id + 1
    else:
      call_g(g)
      id = id + 1
  id_image = id_image + 1   
#save the data in a json file
with open("dataset.json", 'w') as f:
  json.dump(dataset, f)


FileNotFoundError: ignored

another way for downloading detectron:

In [ ]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("my_dataset_train", {}, "dataset.json", "/content/WHU_sample/train/image/")
#register_coco_instances("my_dataset_val", {}, "json_annotation_val.json", "path/to/image/dir")

In [ ]:
from detectron2.engine import DefaultTrainer

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2  # This is the real "batch size" commonly known to deep learning people
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 300    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # The "RoIHead batch size". 128 is faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
#trainer = DefaultTrainer(cfg) 
#trainer.resume_or_load(resume=False)
#trainer.train()

In [7]:
import os
os.environ['LD_LIBRARY_PATH'] = "/usr/local/cuda/lib64"
!git clone https://github.com/aim-uofa/AdelaiDet.git
%cd AdelaiDet
!python setup.py build develop

Cloning into 'AdelaiDet'...
remote: Enumerating objects: 2321, done.
remote: Total 2321 (delta 0), reused 0 (delta 0), pack-reused 2321
Receiving objects: 100% (2321/2321), 625.84 KiB | 12.77 MiB/s, done.
Resolving deltas: 100% (1432/1432), done.
/content/AdelaiDet
/usr/local/lib/python3.7/dist-packages/torch/cuda/__init__.py:82: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 803: system has unsupported display driver / cuda driver combination (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
running build
running build_py
creating build
creating build/lib.linux-x86_64-3.7
creating build/lib.linux-x86_64-3.7/adet
copying adet/__init__.py -> build/lib.linux-x86_64-3.7/adet
creating build/lib.linux-x86_64-3.7/adet/evaluation
copying ade

In [19]:
!nvcc --v
!/usr/local/cuda/bin/nvcc

nvcc fatal   : Unknown option '--v'
nvcc fatal   : No input files specified; use option --help for more information


In [ ]:
# #sys.path.append("/content/AdelaiDet/adet/config")
# print(sys.path)
# sys.path = sys.path[:-7]
# #sys.path.extend(['/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.7/dist-packages/IPython/extensions', '/root/.ipython'])
# print(sys.path)
# #sys.path.clear()
# %cd /content/AdelaiDet/adet/config
# !python config.py build develop
# !python defaults.py build develop
#!python __init__.py build develop
#from adet.config import config
#sys.path.append("/content/AdelaiDet/")
!pip install git+https://github.com/aim-uofa/AdelaiDet.git
#from importlib import import_module
#from config import config.py

In [8]:
#!git clone "https://github.com/aim-uofa/AdelaiDet"
#import adet
#!pip install adet
#sys.path.append("/content/AdelaiDet/adet/config")
#!python config.py build

#from importlib import import_module
#m_module = import_module("/content/AdelaiDet/adet/config/config.py")
#import config
#from AdelaiDet import adet
from adet.config import get_cfg as get_cfg_adet # use the default config from adet instead of the detectron2 one

cfg = get_cfg_adet()
cfg.merge_from_file("/content/AdelaiDet/configs/SOLOv2/R50_3x.yaml") # path to config file (here, it's specified relative to the AdelaiDet root folder but can also be an absolute path)
cfg.MODEL.WEIGHTS = "SOLOv2_R50_3x.pth" # path to the corresponding pre-trained weights, assuming these have been downloaded as described here https://github.com/aim-uofa/AdelaiDet/tree/master/configs/SOLOv2

cfg.DATASETS.TRAIN = ("my_dataset_train",) # name should match the one used when registering the dataset
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2

cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 300    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate

# Override SOLOv2 config settings
cfg.MODEL.SOLOV2.NUM_CLASSES = 1 # only one class for the balloon dataset

In [9]:
print(cfg.MODEL.SOLOV2)

FPN_INSTANCE_STRIDES: [8, 8, 16, 32, 32]
FPN_SCALE_RANGES: ((1, 96), (48, 192), (96, 384), (192, 768), (384, 2048))
INSTANCE_CHANNELS: 512
INSTANCE_IN_CHANNELS: 256
INSTANCE_IN_FEATURES: ['p2', 'p3', 'p4', 'p5', 'p6']
LOSS:
  DICE_WEIGHT: 3.0
  FOCAL_ALPHA: 0.25
  FOCAL_GAMMA: 2.0
  FOCAL_USE_SIGMOID: True
  FOCAL_WEIGHT: 1.0
MASK_CHANNELS: 128
MASK_IN_CHANNELS: 256
MASK_IN_FEATURES: ['p2', 'p3', 'p4', 'p5']
MASK_THR: 0.5
MAX_PER_IMG: 100
NMS_KERNEL: gaussian
NMS_PRE: 500
NMS_SIGMA: 2
NMS_TYPE: matrix
NORM: GN
NUM_CLASSES: 1
NUM_GRIDS: [40, 36, 24, 16, 12]
NUM_INSTANCE_CONVS: 4
NUM_KERNELS: 256
NUM_MASKS: 256
PRIOR_PROB: 0.01
SCORE_THR: 0.1
SIGMA: 0.2
TYPE_DCN: DCN
UPDATE_THR: 0.05
USE_COORD_CONV: True
USE_DCN_IN_INSTANCE: False
